# Предобрабатываю данные (избавляюсь от дубликатов), агрегирую до года, соединяю таблицы

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
cd ..

c:\Users\unstr\Documents\7 семестр работы\Диплом\thesis-repo\thesis\data


c:\Users\unstr\Documents\7 семестр работы\Диплом\thesis-repo\thesis\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [1]:
import polars as pl

In [ ]:
trade19 = pl.read_csv('./raw_data/BACI_HS17_V202501/BACI_HS17_Y2019_V202501.csv')
trade20 = pl.read_csv('./raw_data/BACI_HS17_V202501/BACI_HS17_Y2020_V202501.csv')
trade21 = pl.read_csv('./raw_data/BACI_HS17_V202501/BACI_HS17_Y2021_V202501.csv')
trade22 = pl.read_csv('./raw_data/BACI_HS17_V202501/BACI_HS17_Y2022_V202501.csv')
country_codes = pl.read_csv('./raw_data/BACI_HS17_V202501/country_codes_V202501.csv')
sql = pl.SQLContext(register_globals=True)

In [4]:
#Структура данных. t, i , j , k --- ключ.
sql.execute('''select * from trade19 limit 1;''').collect()

t,i,j,k,v,q
i64,i64,i64,i64,f64,f64
2019,4,20,420229,0.471,0.001


In [5]:
#Проверяю, что в данных нет дублей
print(sql.execute('''select i, j, k, count(*) as cnt from trade19 group by 1, 2, 3 order by 4 desc limit 1;''').collect())
print(sql.execute('''select i, j, k, count(*) as cnt from trade20 group by 1, 2, 3 order by 4 desc limit 1;''').collect())
print(sql.execute('''select i, j, k, count(*) as cnt from trade21 group by 1, 2, 3 order by 4 desc limit 1;''').collect())
print(sql.execute('''select i, j, k, count(*) as cnt from trade22 group by 1, 2, 3 order by 4 desc limit 1;''').collect())

shape: (1, 4)
┌─────┬─────┬────────┬─────┐
│ i   ┆ j   ┆ k      ┆ cnt │
│ --- ┆ --- ┆ ---    ┆ --- │
│ i64 ┆ i64 ┆ i64    ┆ u32 │
╞═════╪═════╪════════╪═════╡
│ 246 ┆ 458 ┆ 621149 ┆ 1   │
└─────┴─────┴────────┴─────┘
shape: (1, 4)
┌─────┬─────┬────────┬─────┐
│ i   ┆ j   ┆ k      ┆ cnt │
│ --- ┆ --- ┆ ---    ┆ --- │
│ i64 ┆ i64 ┆ i64    ┆ u32 │
╞═════╪═════╪════════╪═════╡
│ 170 ┆ 528 ┆ 620192 ┆ 1   │
└─────┴─────┴────────┴─────┘
shape: (1, 4)
┌─────┬─────┬────────┬─────┐
│ i   ┆ j   ┆ k      ┆ cnt │
│ --- ┆ --- ┆ ---    ┆ --- │
│ i64 ┆ i64 ┆ i64    ┆ u32 │
╞═════╪═════╪════════╪═════╡
│ 724 ┆ 251 ┆ 600192 ┆ 1   │
└─────┴─────┴────────┴─────┘
shape: (1, 4)
┌─────┬─────┬────────┬─────┐
│ i   ┆ j   ┆ k      ┆ cnt │
│ --- ┆ --- ┆ ---    ┆ --- │
│ i64 ┆ i64 ┆ i64    ┆ u32 │
╞═════╪═════╪════════╪═════╡
│ 764 ┆ 56  ┆ 551599 ┆ 1   │
└─────┴─────┴────────┴─────┘


In [4]:
# Меняю типы данных на более экономные, где это возможно, и объединяю таблицы для трёх лет
trade_tmp = sql.execute(''' 
            select 
                    t::smallint
                    , i::smallint
                    , j::smallint
                    , k
                    , v::float4
                    , q::float4
                    ,  case when length(k::varchar) = 5 then '0'||(k::varchar) 
                            else k::varchar end as category 
                    from trade20
            union all
            select 
                    t::smallint
                    , i::smallint
                    , j::smallint
                    , k
                    , v::float4
                    , q::float4
                    ,  case when length(k::varchar) = 5 then '0'||(k::varchar) 
                            else k::varchar end as category 
                    from trade21
            union all
            select 
                    t::smallint
                    , i::smallint
                    , j::smallint
                    , k
                    , v::float4
                    , q::float4
                    ,  case when length(k::varchar) = 5 then '0'||(k::varchar) 
                            else k::varchar end as category 
                    from trade22
            ;
            ''').collect()
sql.register('trade_tmp', trade_tmp)
sql.execute('''select count(*) from trade_tmp;''').collect()

len
u32
34145929


In [5]:
# Удаляю лишние таблицы, чтобы не засоряли оперативку
import gc
del trade20
del trade21
del trade22
gc.collect()

23

In [6]:
# Джоиню данные за 2019 год, чтобы было потом на них нормировать
trade_tmp2 = sql.execute('''
            select 
                tr.t
                , tr.i
                , tr.j
                , tr.category    
                , tr.v
                , tr.q             
                , t19.v as v19
                , t19.q as q19    
            from trade_tmp tr
            inner join trade19 t19
                on tr.i = t19.i
                and tr.j = t19.j
                and tr.k = t19.k
            ;''').collect()
sql.register('trade_tmp2', trade_tmp2)
del trade19
del trade_tmp
gc.collect()

0

In [ ]:
# Материализовал тяжёлый запрос, чтобы больше не мучить компьютер
# trade_tmp2.write_parquet('preprocessed_data/trade_tmp2.parquet.gzip')

In [8]:
# Хотя в country_codes есть страны, у которых два типа country_code, но один iso3, это соответствует кейсам преемственности. 
# На интервале 2019 -- 2022 этого не возникает (т.е. можно считать, что iso3 --- PK)
sql.execute('''select country_iso3, count(*) from country_codes group by 1, order by 2 desc limit 4;''').collect()

country_iso3,len
str,u32
"""BEL""",2
"""DEU""",2
"""SDN""",2
"""KWT""",1


In [9]:
# У некоторых стран два country_code, но один iso3 код. Чтобы убрать дубли, группирую и суммирую
trade = sql.execute('''
    select 
        tr.t
        , cc_i.country_iso3 as i
        , cc_j.country_iso3 as j
        , tr.category    
        , v
        , q             
        , v19
        , q19
    from trade_tmp2 tr
    inner join country_codes cc_i
        on tr.i = cc_i.country_code 
    inner join country_codes cc_j
        on tr.j = cc_j.country_code                      
;''').collect()
sql.register('trade', trade)
sql.execute('''select * from trade limit 2;''').collect()

t,i,j,category,v,q,v19,q19
i16,str,str,str,f32,f32,f64,f64
2020,"""AFG""","""AUS""","""080211""",2.456,0.71,6.944,1.198
2020,"""AFG""","""AUS""","""080420""",1.398,1.331,0.87,0.8


In [10]:
trade.write_parquet('preprocessed_data/trade_hs6.parquet.gzip')

In [ ]:
# Агрегирую данные о торговле с разной гранулярностью гармонизированной системы
sql.execute('''
            select 
                t, i, j, left(category, 4) as category
                , sum(v) as v
                , sum(q) as q
                , sum(v19) as v19
                , sum(q19) as q19
            from trade
            group by 1, 2, 3, 4
            ;''').collect().write_parquet('preprocessed_data/trade_hs4.parquet.gzip')

In [12]:
sql.execute('''select 
                t, i, j, left(category, 2) as category
                , sum(v) as v
                , sum(q) as q
                , sum(v19) as v19
                , sum(q19) as q19
            from trade
            group by 1, 2, 3, 4
            ;''').collect().write_parquet('preprocessed_data/trade_hs2.parquet.gzip')

In [13]:
sql.execute('''select 
                t, i, j
                , sum(v) as v
                , sum(q) as q
                , sum(v19) as v19
                , sum(q19) as q19
            from trade
            group by 1, 2, 3
            ;''').collect().write_parquet('preprocessed_data/trade_hs0.parquet.gzip')

In [14]:
del trade_tmp2
del trade
gc.collect()

21

## Регрессоры и инструменты

### Заболеваемость и смертность

In [27]:
covid_tmp = pl.read_csv('raw_data/Our World In Data/owid-covid-data.csv')
sql.register('covid_tmp', covid_tmp)
sql.execute('''select * from covid_tmp limit 2;''').collect()

iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred,stringency_index,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
str,str,str,str,i64,i64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,str,f64,f64,str,str,f64,f64,f64,f64,i64,str,str,str,str
"""AFG""","""Asia""","""Afghanistan""","""2020-01-05""",0,0,null,0,0,null,0.0,0.0,null,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,54.42,18.6,2.58,1.34,1803.99,null,597.03,9.59,null,null,37.75,0.5,64.83,0.51,41128772,null,null,null,null
"""AFG""","""Asia""","""Afghanistan""","""2020-01-06""",0,0,null,0,0,null,0.0,0.0,null,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,54.42,18.6,2.58,1.34,1803.99,null,597.03,9.59,null,null,37.75,0.5,64.83,0.51,41128772,null,null,null,null


In [56]:
# Надо подумать, нет ли тут эндогенных пропусков (т.е. если у страны нулевая заболеваемость,
#  значит ли это, что она ничего не репортит, а не репортит 0)
sql.execute('''create table covid as
    select 
        iso_code as country
        , left(date, 4)::smallint as t
        , avg(new_cases_per_million) as cases
        , avg(new_deaths_per_million) as deaths
        , avg(new_tests_per_thousand) as tests
        /*Лучше воспрользоваться данными ООН, чтобы точно использовать доковидные данные
        , avg(population_density) as population_density
        , avg(median_age) as median_age 
        , avg(aged_65_older) as aged_65_older
        , avg(aged_70_older) as aged_70_older
        , avg(cardiovasc_death_rate) as cardiovasc_death_rate
        , avg(handwashing_facilities) as handwashing_facilities
        , avg(life_expectancy) as life_expectancy */
    from covid_tmp
    where date <= '2022-12-31'
    group by iso_code, left(date, 4)        
;''')
sql.execute('''select * from covid limit 2;''').collect()

country,t,cases,deaths,tests
str,i16,f64,f64,str
"""BTN""",2021,7.237644,0.010521,null
"""PNG""",2022,2.836027,0.02126,null


### Коронавирусные ограничения

In [29]:
restrictions_tmp = pl.read_csv('raw_data/OxCGRT/OxCGRT_compact_national_v1.csv')
sql.register('restrictions_tmp', restrictions_tmp)
sql.execute('''select * from restrictions_tmp 
            -- where `E3_Fiscal measures` is not null and `E3_Fiscal measures` > 0 
            limit 2;''').collect()

CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1M_School closing,C1M_Flag,C2M_Workplace closing,C2M_Flag,C3M_Cancel public events,C3M_Flag,C4M_Restrictions on gatherings,C4M_Flag,C5M_Close public transport,C5M_Flag,C6M_Stay at home requirements,C6M_Flag,C7M_Restrictions on internal movement,C7M_Flag,C8EV_International travel controls,E1_Income support,E1_Flag,E2_Debt/contract relief,E3_Fiscal measures,E4_International support,H1_Public information campaigns,H1_Flag,H2_Testing policy,H3_Contact tracing,H4_Emergency investment in healthcare,H5_Investment in vaccines,H6M_Facial Coverings,H6M_Flag,H7_Vaccination policy,H7_Flag,H8M_Protection of elderly people,H8M_Flag,V1_Vaccine Prioritisation (summary),V2A_Vaccine Availability (summary),V2B_Vaccine age eligibility/availability age floor (general population summary),V2C_Vaccine age eligibility/availability age floor (at risk summary),V2D_Medically/ clinically vulnerable (Non-elderly),V2E_Education,V2F_Frontline workers (non healthcare),V2G_Frontline workers (healthcare),V3_Vaccine Financial Support (summary),V4_Mandatory Vaccination (summary),ConfirmedCases,ConfirmedDeaths,MajorityVaccinated,PopulationVaccinated,StringencyIndex_Average,GovernmentResponseIndex_Average,ContainmentHealthIndex_Average,EconomicSupportIndex
str,str,str,str,str,i64,f64,i64,f64,i64,f64,i64,f64,i64,f64,str,f64,i64,f64,i64,f64,f64,i64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,str,f64,str,f64,i64,i64,i64,str,str,str,str,str,str,i64,str,i64,i64,str,f64,f64,f64,f64,f64
"""Aruba""","""ABW""",null,null,"""NAT_TOTAL""",20200101,0.0,null,0.0,null,0.0,null,0.0,null,0.0,null,0.0,null,0.0,null,0.0,0.0,null,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,null,0.0,null,0.0,null,0,0,null,null,null,null,null,null,0,null,0,0,"""NV""",0.0,0.0,0.0,0.0,0.0
"""Aruba""","""ABW""",null,null,"""NAT_TOTAL""",20200102,0.0,null,0.0,null,0.0,null,0.0,null,0.0,null,0.0,null,0.0,null,0.0,0.0,null,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,null,0.0,null,0.0,null,0,0,null,null,null,null,null,null,0,null,0,0,"""NV""",0.0,0.0,0.0,0.0,0.0


In [31]:
sql.execute('''create table restrictions as
    select 
        CountryCode as country
        , left(Date::varchar, 4)::smallint as t
        , avg(`C1M_School closing`)::float4 as C1M_School_closing
        , avg(`C2M_Workplace closing`)::float4 as C2M_Workplace_closing
        , avg(`C3M_Cancel public events`)::float4 as C3M_Cancel_public_events
        , avg(`C4M_Restrictions on gatherings`)::float4 as C4M_Restrictions_on_gatherings
        , avg(`C5M_Close public transport`)::float4 as C5M_Close_public_transport
        , avg(`C6M_Stay at home requirements`)::float4 as C6M_Stay_at_home_requirements
        , avg(`C7M_Restrictions on internal movement`)::float4 as C7M_Restrictions_on_internal_movement                              
        , avg(`C8EV_International travel controls`)::float4 as C8EV_International_travel_controls
        , avg(`E1_Income support`)::float4 as E1_Income_support
        , avg(`E2_Debt/contract relief`)::float4 as E2_Debt_contract_relief
        , avg(`E3_Fiscal measures`)::float4 as E3_Fiscal_measures
        , avg(`E4_International support`)::float4 as E4_International_support
        , avg(`H1_Public information campaigns`)::float4 as H1_Public_information_campaigns	
        , avg(`H2_Testing policy`)::float4 as H2_Testing_policy
        , avg(`H3_Contact tracing`)::float4 as H3_Contact_tracing
        , avg(`H4_Emergency investment in healthcare`)::float4 as H4_Emergency_investment_in_healthcare
        , avg(`H5_Investment in vaccines`)::float4 as H5_Investment_in_vaccines
        , avg(`H6M_Facial Coverings`)::float4 as H6M_Facial_Coverings
        , avg(`H7_Vaccination policy`)::float4 as H7_Vaccination_policy
        , avg(`H8M_Protection of elderly people`)::float4 as H8M_Protection_of_elderly_people
        , avg(`StringencyIndex_Average`)::float4 as StringencyIndex_Average
        , avg(`GovernmentResponseIndex_Average`)::float4 as GovernmentResponseIndex_Average
        , avg(`ContainmentHealthIndex_Average`)::float4 as ContainmentHealthIndex_Average
        , avg(`EconomicSupportIndex`)::float4 as EconomicSupportIndex
        , avg(case when MajorityVaccinated = 'V' then 1 
                   else 0 end)::float4 as MajorityVaccinated
        , avg(PopulationVaccinated)::float4 as PopulationVaccinated        
    from restrictions_tmp
    group by CountryCode, left(Date::varchar, 4)::smallint           
;''')
sql.execute('''select * from restrictions limit 10;''').collect()

country,t,C1M_School_closing,C2M_Workplace_closing,C3M_Cancel_public_events,C4M_Restrictions_on_gatherings,C5M_Close_public_transport,C6M_Stay_at_home_requirements,C7M_Restrictions_on_internal_movement,C8EV_International_travel_controls,E1_Income_support,E2_Debt_contract_relief,E3_Fiscal_measures,E4_International_support,H1_Public_information_campaigns,H2_Testing_policy,H3_Contact_tracing,H4_Emergency_investment_in_healthcare,H5_Investment_in_vaccines,H6M_Facial_Coverings,H7_Vaccination_policy,H8M_Protection_of_elderly_people,StringencyIndex_Average,GovernmentResponseIndex_Average,ContainmentHealthIndex_Average,EconomicSupportIndex,MajorityVaccinated,PopulationVaccinated
str,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""DOM""",2021,2.136986,1.567123,1.90137,3.983562,0.849315,1.569863,0.29863,1.380822,0.589041,0.0,1.9299e6,0.0,1.783562,2.621918,1.0,450450.4375,2157.925049,3.347945,3.709589,2.463014,60.448685,59.022686,65.815262,11.472603,0.079452,24.325588
"""LKA""",2021,2.230137,1.967123,1.967123,3.827397,1.172603,1.956164,1.657534,2.142466,0.0,2.0,0.0,0.0,2.0,2.994521,2.0,0.0,719178.0625,3.383562,3.720548,1.956164,72.744797,73.395592,76.737343,50.0,0.287671,21.986658
"""UZB""",2022,0.293151,0.216438,0.693151,0.819178,0.0,0.0,0.263014,0.843836,0.235616,0.528767,null,null,1.567123,1.786301,0.0,null,0.0,1.230137,5.0,0.161644,20.527315,26.165342,27.171124,19.109589,0.172603,43.971889
"""UGA""",2022,0.364384,2.0,0.583562,0.958904,1.934247,0.131507,0.021918,1.0,0.0,0.191781,null,null,1.778082,3.0,0.024658,null,0.0,3.753425,4.934247,0.257534,34.912987,38.661396,43.499207,4.79452,0.0,20.547398
"""PER""",2020,2.295082,1.715847,1.612022,3.20765,1.224044,1.743169,1.374317,2.846995,0.795082,1.456284,3.4251532e7,0.0,1.650273,1.653005,1.385246,1.332446e6,0.0,2.412568,0.0,1.811475,67.374481,60.129646,60.679726,56.284153,0.0,0.0
"""ZAF""",2021,1.169863,1.317808,1.284932,2.986301,1.0,1.989041,0.153425,1.221918,0.923288,1.413699,0.0,0.0,2.0,2.739726,2.0,0.0,0.0,3.019178,2.920548,2.235616,52.714439,61.93959,62.440548,58.424656,0.0,7.596795
"""SGP""",2022,1.0,0.756164,0.657534,1.260274,0.0,0.315068,0.0,1.068493,2.0,2.0,null,null,2.0,3.0,2.0,null,0.0,2.476712,5.0,3.0,30.077179,58.289234,52.32811,100.0,1.0,88.951096
"""FRO""",2022,0.158904,0.084932,0.084932,0.282192,0.0,0.0,0.0,0.136986,0.0,0.0,null,null,2.0,1.246575,0.208219,null,0.0,0.0,5.0,0.063014,13.649616,17.304356,19.782576,0.0,1.0,76.959892
"""UGA""",2021,2.323288,2.0,1.389041,2.106849,1.060274,1.942466,1.20274,1.383562,0.408219,0.0,0.0,0.0,2.0,2.690411,0.356164,0.0,13698.629883,2.978082,2.216438,1.772603,63.781452,54.792988,61.161892,10.20548,0.0,0.501425


### Температура

In [32]:
temperature_tmp = pl.read_csv('raw_data/Our World In Data/average-monthly-surface-temperature.csv')
sql.register('temperature_tmp', temperature_tmp)
sql.execute('''select * from temperature_tmp limit 2;''').collect()

Entity,Code,year,Day,Average surface temperature,Average surface temperature_duplicated_0
str,str,i64,str,f64,f64
"""Afghanistan""","""AFG""",1940,"""1940-01-15""",-2.032494,11.327695
"""Afghanistan""","""AFG""",1940,"""1940-02-15""",-0.733503,11.327695


In [57]:
sql.execute('''create table temperature as
    select 
        Code as country
        , year t   
        , avg(`Average surface temperature`) as average_surface_temperature
    from temperature_tmp
    where year >= 2020 --and year <= 2019
    group by Code, year               
;''')
sql.execute('''select * from temperature limit 2;''').collect()

country,t,average_surface_temperature
str,i64,f64
"""CHN""",2020,7.606333
"""CYP""",2024,20.615953


### Median age

In [58]:
median_age_tmp = pl.read_excel('raw_data/United Nations/WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT.xlsx')
sql.register('median_age_tmp', median_age_tmp)
sql.execute('''select * from median_age_tmp where `ISO3 Alpha-code` <> '' limit 2;''').collect()

Could not determine dtype for column 5, falling back to string
Could not determine dtype for column 6, falling back to string


Index,Variant,"Region, subregion, country or area *",Notes,Location code,ISO3 Alpha-code,ISO2 Alpha-code,SDMX code**,Type,Parent code,Year,"Total Population, as of 1 January (thousands)","Total Population, as of 1 July (thousands)","Male Population, as of 1 July (thousands)","Female Population, as of 1 July (thousands)","Population Density, as of 1 July (persons per square km)","Population Sex Ratio, as of 1 July (males per 100 females)",MedianAge1July,"Natural Change, Births minus Deaths (thousands)","Rate of Natural Change (per 1,000 population)",Population Change (thousands),Population Growth Rate (percentage),Population Annual Doubling Time (years),Births (thousands),Births by women aged 15 to 19 (thousands),"Crude Birth Rate (births per 1,000 population)",Total Fertility Rate (live births per woman),Net Reproduction Rate (surviving daughters per woman),Mean Age Childbearing (years),Sex Ratio at Birth (males per 100 female births),Total Deaths (thousands),Male Deaths (thousands),Female Deaths (thousands),"Crude Death Rate (deaths per 1,000 population)","Life Expectancy at Birth, both sexes (years)",Male Life Expectancy at Birth (years),Female Life Expectancy at Birth (years),"Life Expectancy at Age 15, both sexes (years)",Male Life Expectancy at Age 15 (years),Female Life Expectancy at Age 15 (years),"Life Expectancy at Age 65, both sexes (years)",Male Life Expectancy at Age 65 (years),Female Life Expectancy at Age 65 (years),"Life Expectancy at Age 80, both sexes (years)",Male Life Expectancy at Age 80 (years),Female Life Expectancy at Age 80 (years),"Infant Deaths, under age 1 (thousands)","Infant Mortality Rate (infant deaths per 1,000 live births)",Live Births Surviving to Age 1 (thousands),"Under-Five Deaths, under age 5 (thousands)","Under-Five Mortality (deaths under age 5 per 1,000 live births)","Mortality before Age 40, both sexes (deaths under age 40 per 1,000 live births)","Male Mortality before Age 40 (deaths under age 40 per 1,000 male live births)","Female Mortality before Age 40 (deaths under age 40 per 1,000 female live births)","Mortality before Age 60, both sexes (deaths under age 60 per 1,000 live births)","Male Mortality before Age 60 (deaths under age 60 per 1,000 male live births)","Female Mortality before Age 60 (deaths under age 60 per 1,000 female live births)","Mortality between Age 15 and 50, both sexes (deaths under age 50 per 1,000 alive at age 15)","Male Mortality between Age 15 and 50 (deaths under age 50 per 1,000 males alive at age 15)","Female Mortality between Age 15 and 50 (deaths under age 50 per 1,000 females alive at age 15)","Mortality between Age 15 and 60, both sexes (deaths under age 60 per 1,000 alive at age 15)","Male Mortality between Age 15 and 60 (deaths under age 60 per 1,000 males alive at age 15)","Female Mortality between Age 15 and 60 (deaths under age 60 per 1,000 females alive at age 15)",Net Number of Migrants (thousands),"Net Migration Rate (per 1,000 population)"
i64,str,str,str,i64,str,str,i64,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
2595,"""Estimates""","""Burundi""","""""",108,"""BDI""","""BI""",108,"""Country/Area""",910,1950,"""2229.322""","""2254.938""","""1080.184""","""1174.755""","""86.895""","""91.95""","""18.305""","""64.551""","""28.627""","""51.232""","""2.272""","""30.508""","""117.211""","""11.582""","""51.98""","""6.923""","""2.072""","""30.674""","""102.5""","""52.66""","""26.76""","""25.899""","""23.353""","""40.938""","""39.571""","""42.245""","""44.916""","""43.636""","""46.095""","""11.048""","""10.594""","""11.423""","""4.728""","""4.454""","""4.937""","""17.777""","""162.866""","""104.421""","""28.358""","""276.13""","""457.62""","""471.554""","""444.227""","""614.352""","""639.661""","""590.518""","""290.807""","""309.954""","""273.27""","""423.97""","""454.7

In [59]:
sql.execute('''create table median_age_table as
    select 
        `ISO3 Alpha-code` as country
        , Year as t
        , avg(MedianAge1July::float4) as median_age
    from median_age_tmp
    where `ISO3 Alpha-code` <> '' 
            and Year >= 2020       
    group by `ISO3 Alpha-code`, Year       
;''')
sql.execute('''select * from median_age_table limit 2;''').collect()

country,t,median_age
str,i64,f32
"""URY""",2022,35.495998
"""BGR""",2021,44.042999


### SARS

In [60]:
SARS_table_tmp = pl.read_excel('raw_data/WHO/SARS.xlsx')
sql.register('SARS_table_tmp', SARS_table_tmp)
sql.execute('''create table SARS_table as 
            select iso3 as country, Total as SARS_total_cases 
            from SARS_table_tmp;''').collect()
sql.execute('''select * from SARS_table limit 2;''').collect()

country,SARS_total_cases
str,i64
"""AUT""",6
"""CAN""",251


### Плотность населения и смертность

In [61]:
density_and_mortality_tmp = pl.read_csv('raw_data/World Bank/pop_dens_and_mortality.csv')
sql.register('density_and_mortality_tmp', density_and_mortality_tmp)
sql.execute('''select * from density_and_mortality_tmp limit 2;''').collect()

Time,Time Code,Country Name,Country Code,Population density (people per sq. km of land area) [EN.POP.DNST],"Population, female (% of total population) [SP.POP.TOTL.FE.ZS]","Maternal mortality ratio (national estimate, per 100,000 live births) [SH.STA.MMRT.NE]","Mortality from CVD, cancer, diabetes or CRD between exact ages 30 and 70 (%) [SH.DYN.NCOM.ZS]","Mortality rate attributed to household and ambient air pollution, age-standardized (per 100,000 population) [SH.STA.AIRP.P5]","Mortality rate attributed to unintentional poisoning (per 100,000 population) [SH.STA.POIS.P5]","Mortality rate attributed to unsafe water, unsafe sanitation and lack of hygiene (per 100,000 population) [SH.STA.WASH.P5]","Mortality rate, adult, female (per 1,000 female adults) [SP.DYN.AMRT.FE]","Mortality rate, adult, male (per 1,000 male adults) [SP.DYN.AMRT.MA]","Mortality rate, infant (per 1,000 live births) [SP.DYN.IMRT.IN]",Population in the largest city (% of urban population) [EN.URB.LCTY.UR.ZS],Population in urban agglomerations of more than 1 million (% of total population) [EN.URB.MCTY.TL.ZS]
i64,str,str,str,f64,f64,i64,str,str,str,str,f64,f64,f64,f64,f64
1990,"""YR1990""","""Afghanistan""","""AFG""",18.468424,50.096823,null,null,null,null,null,382.051,510.052,122.5,60.735996,12.86206
1990,"""YR1990""","""Albania""","""ALB""",119.946788,49.765495,null,null,null,null,null,66.103,127.21,35.2,20.653647,null


In [63]:
sql.execute('''create table density_and_mortality as
    select 
        `Country Code` as country
        , Time as t
        , avg(`Population density (people per sq. km of land area) [EN.POP.DNST]`) population_density
        , avg(`Population, female (% of total population) [SP.POP.TOTL.FE.ZS]`) female_ratio
        , avg(`Maternal mortality ratio (national estimate, per 100,000 live births) [SH.STA.MMRT.NE]`)::float4 maternal_mortality
        , avg(`Mortality from CVD, cancer, diabetes or CRD between exact ages 30 and 70 (%) [SH.DYN.NCOM.ZS]`)::float4 cvd_mortality
        , avg(`Mortality rate attributed to household and ambient air pollution, age-standardized (per 100,000 population) [SH.STA.AIRP.P5]`)::float4 pollution_mortality
        , avg(`Mortality rate attributed to unintentional poisoning (per 100,000 population) [SH.STA.POIS.P5]`)::float4 poisoning_mortality 
        , avg(`Mortality rate attributed to unsafe water, unsafe sanitation and lack of hygiene (per 100,000 population) [SH.STA.WASH.P5]`)::float4 hygiene_mortality
        , avg(`Mortality rate, adult, female (per 1,000 female adults) [SP.DYN.AMRT.FE]`)::float4 female_mortality
        , avg(`Mortality rate, adult, male (per 1,000 male adults) [SP.DYN.AMRT.MA]`)::float4 male_mortality
        , avg(`Mortality rate, infant (per 1,000 live births) [SP.DYN.IMRT.IN]`)::float4 infant_mortality
        , avg(`Population in the largest city (% of urban population) [EN.URB.LCTY.UR.ZS]`)::float4 largest_city_share
        , avg(`Population in urban agglomerations of more than 1 million (% of total population) [EN.URB.MCTY.TL.ZS]`)::float4 urbanization 
    from density_and_mortality_tmp
    where Time >= 2020        
    group by `Country Code`, Time
;''')
sql.execute('''select * from density_and_mortality order by country, t limit 10;''').collect()

country,t,population_density,female_ratio,maternal_mortality,cvd_mortality,pollution_mortality,poisoning_mortality,hygiene_mortality,female_mortality,male_mortality,infant_mortality,largest_city_share,urbanization
str,i64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""ABW""",2020,603.261111,52.844507,null,null,null,null,null,59.681999,124.098999,null,null,null
"""ABW""",2021,598.333333,52.83196,null,null,null,null,null,70.364998,140.466003,null,null,null
"""ABW""",2022,596.166667,52.792674,null,null,null,null,null,67.945999,137.216003,null,null,null
"""ABW""",2023,null,52.76962,null,null,null,null,null,null,null,null,null,null
"""AFE""",2020,46.779514,50.525117,null,null,null,null,null,224.25235,311.401581,42.858887,27.929363,14.379939
"""AFE""",2021,48.035473,50.519416,null,null,null,null,null,237.652527,332.734894,41.823822,27.894281,14.508222
"""AFE""",2022,49.297201,50.51171,null,null,null,null,null,233.05043,326.323669,40.797623,27.883209,14.649723
"""AFE""",2023,null,50.502844,null,null,null,null,null,null,null,null,27.901375,14.812855
"""AFG""",2020,59.900616,49.511222,null,null,null,null,null,210.052994,318.587006,47.400002,41.517445,10.805329


## Переменные, которые не входят в первый приоритет

In [64]:
household_size_tmp = pl.read_excel('raw_data/United Nations/undesa_pd_2022_hh-size-composition.xlsx')
sql.register('household_size_tmp', household_size_tmp)
sql.execute('''select * from household_size_tmp limit 2;''').collect()

Could not determine dtype for column 9, falling back to string
Could not determine dtype for column 10, falling back to string


Country or area,ISO Code,Data source category,Reference date (dd/mm/yyyy),Average household size (number of members),1 member,2-3 members,4-5 members,6 or more members
str,i64,str,str,f64,f64,f64,f64,f64
"""Afghanistan""",4,"""MICS""","""2010-01-07 00:00:00""",7.75,0.15,7.41,18.59,73.85
"""Afghanistan""",4,"""DHS""","""19/10/2015""",8.04,0.19,6.45,16.84,76.52


In [65]:
sql.execute('''create table household_size as
            select country, avg_hh_size, share_1_members, share_2_3_members, share_4_5_members
            from
            (select distinct on(country)
                cc.country_iso3 as country
                , case when `Reference date (dd/mm/yyyy)` like '__/__/____' then right(`Reference date (dd/mm/yyyy)`, 4)
                    else left(`Reference date (dd/mm/yyyy)`, 4) end ::int as year 
                , `Data source category` as data_source
                , `Average household size (number of members)`::float4 avg_hh_size
                , `1 member`::float4 as share_1_members
                , `2-3 members`::float4 as share_2_3_members
                , `4-5 members`::float4 as share_4_5_members
            from household_size_tmp hh
            inner join country_codes cc
                on hh.`ISO Code` = cc.country_code
            order by 1, 2 desc
            , data_source -- для детерминированности
            ) sub
;
''')
sql.execute('''select * from household_size order by 3 desc limit 10;''').collect()

country,avg_hh_size,share_1_members,share_2_3_members,share_4_5_members
str,f32,f32,f32,f32
"""AUS""",2.55,null,null,null
"""JAM""",3.06,null,null,null
"""KNA""",null,null,null,null
"""NZL""",2.6,null,null,null
"""LVA""",2.26,41.080002,40.799999,14.41
"""FIN""",2.07,41.009998,44.459999,12.97
"""BLR""",2.16,37.959999,47.02,13.54
"""DEU""",2.14,37.27,48.09,13.21
"""AUT""",2.27,36.290001,44.93,16.49


## Соединяю информацию о коронавирусе, ограничениях и контрольных переменных в одну таблицу и экспортирую её

In [66]:
sql.register('regressors_tmp',
            sql.execute('''
    select 
        *
    from covid c
    full outer join restrictions r
        using(country, t)
    ;''').collect().with_columns(pl.coalesce(['country', 'country:r']).alias("country")
                                    , pl.coalesce(['t', 't:r']).alias("t")).drop(['country:r', 't:r']))
regressors = sql.execute('''
    select * 
    from regressors_tmp r
    left join temperature temp
        using(country, t)
    left join median_age_table
        using(country, t) 
    left join SARS_table
        using(country)
    left join density_and_mortality
        using(country, t) 
    left join household_size
        using(country)
    ;''').collect()
regressors

country,t,cases,deaths,tests,C1M_School_closing,C2M_Workplace_closing,C3M_Cancel_public_events,C4M_Restrictions_on_gatherings,C5M_Close_public_transport,C6M_Stay_at_home_requirements,C7M_Restrictions_on_internal_movement,C8EV_International_travel_controls,E1_Income_support,E2_Debt_contract_relief,E3_Fiscal_measures,E4_International_support,H1_Public_information_campaigns,H2_Testing_policy,H3_Contact_tracing,H4_Emergency_investment_in_healthcare,H5_Investment_in_vaccines,H6M_Facial_Coverings,H7_Vaccination_policy,H8M_Protection_of_elderly_people,StringencyIndex_Average,GovernmentResponseIndex_Average,ContainmentHealthIndex_Average,EconomicSupportIndex,MajorityVaccinated,PopulationVaccinated,country:temp,t:temp,average_surface_temperature,country:median_age_table,t:median_age_table,median_age,country:SARS_table,SARS_total_cases,country:density_and_mortality,t:density_and_mortality,population_density,female_ratio,maternal_mortality,cvd_mortality,pollution_mortality,poisoning_mortality,hygiene_mortality,female_mortality,male_mortality,infant_mortality,largest_city_share,urbanization,country:household_size,avg_hh_size,share_1_members,share_2_3_members,share_4_5_members
str,i16,f64,f64,str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,i64,f64,str,i64,f32,str,i64,str,i64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,str,f32,f32,f32,f32
"""SLV""",2020,19.625912,0.570608,null,2.426229,1.434426,1.306011,2.297814,0.833333,1.800546,1.295082,2.70765,0.751366,1.445355,4.871233e6,0.0,1.879781,0.808743,0.631148,489863.0,0.0,2.196721,0.0,2.046448,63.594536,53.958168,53.822815,54.918034,0.0,0.0,"""SLV""",2020,26.48445,"""SLV""",2020,25.691,null,null,"""SLV""",2020,300.901207,52.477556,null,null,null,null,null,124.344002,293.265015,11.0,24.148672,17.73575,"""SLV""",3.92,8.46,36.009998,37.84
"""ASM""",2022,467.620027,1.926,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""ASM""",2022,26.570254,"""ASM""",2022,27.927,null,null,"""ASM""",2022,241.71,49.464234,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""BOL""",2020,35.130884,2.077541,null,2.418033,1.830601,1.527322,2.844262,1.420765,1.510929,1.478142,2.202186,0.754098,0.251366,3.198088e6,0.0,1.614754,0.734973,0.581967,340103.40625,0.0,1.904372,0.0,1.9153,67.19986,51.258606,54.988361,25.136612,0.0,0.0,"""BOL""",2020,21.083227,"""BOL""",2020,24.02,null,null,"""BOL""",2020,10.907689,49.729903,null,null,null,null,null,190.705002,316.665009,21.0,22.421068,41.251427,"""BOL""",3.53,20.34,34.82,27.639999
"""SLE""",2020,0.850718,0.025304,null,1.606557,0.234973,1.087432,1.822404,0.336066,0.36612,0.349727,1.770492,0.0,0.418033,20491.802734,0.0,1.489071,1.508197,1.508197,10109.290039,0.0,2.081967,0.0,1.519126,35.848171,35.743771,39.356613,10.45082,0.0,0.0,"""SLE""",2020,26.401541,"""SLE""",2020,18.605,null,null,"""SLE""",2020,109.622582,50.152719,null,null,null,null,null,228.945999,270.278992,80.300003,35.388439,15.189778,"""SLE""",5.29,6.42,20.52,33.279999
"""LKA""",2020,4.966492,0.023094,null,1.931694,0.554645,1.027322,2.180328,0.45082,1.543716,0.775956,2.308743,0.0,1.327869,2460477.5,370076.5,1.726776,1.636612,1.803279,365565.84375,0.0,2.469945,0.0,1.434426,47.475082,46.428089,48.318417,33.19672,0.0,0.0,"""LKA""",2020,26.989581,"""LKA""",2020,32.074001,null,null,"""LKA""",2020,354.332363,51.572804,null,null,null,null,null,65.980003,152.108994,6.0,14.933679,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""MKD""",2021,210.683233,8.122795,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""MKD""",2021,11.432134,"""MKD""",2021,39.647999,null,null,"""MKD""",2021,72.843537,51.352433,null,null,null,null,null,73.846001,128.363998,4.6,55.622864,null,"""

In [67]:
regressors.write_parquet('preprocessed_data/regressors.parquet.gzip')